In [ ]:
import requests
import json
from datetime import datetime, timedelta
import random

# Configuration

# Read the keys from agroscope_keys.json
with open('configs.json', 'r') as f:
    configs = json.load(f)

PROJECT_ID = configs['PROJECT_ID']
BASE_URL = configs['BASE_URL']
MASTER_KEY = configs['master']
WRITE_KEY = configs['read']
READ_KEY = configs['write']

In [ ]:
BASE_URL

In [ ]:
print("🌱 Nostradamus IoTO API - Soil Data Example")

## Step 1: Create Soil Collection

def create_soil_collection(project_id, master_key):
    """Create soil data collection"""
    url = f"{BASE_URL}/projects/{project_id}/collections"
    headers = {"X-API-Key": master_key}

    collection_data = {
        "name": f"sensors",
        "description": "Soil monitoring data",
        "tags": ["soil", "agriculture", "sensors"],
        "collection_schema": {
            "key": "SOIL_001",                    # Identifying key
            "timestamp": "2025-06-17T10:30:00Z",        # Timestamp
            "soil_moisture": 45.2,                      # Soil moisture %
            "ph_level": 6.8,                           # pH level
            "temperature": 18.5,                        # Temperature °C
            "nitrogen": 45.0,                          # Nitrogen ppm
            "phosphorus": 12.5,                        # Phosphorus ppm
            "potassium": 180.0,                        # Potassium ppm
            "battery_level": 87.3                      # Battery %
        }
    }

    response = requests.post(url, json=collection_data, headers=headers)
    if response.status_code == 200:
        print("✅ Collection created successfully!")
        return response.json()
    else:
        print(f"Failed to create collection: {response.text}")
        return None

collection_result = create_soil_collection(PROJECT_ID, MASTER_KEY)
print(collection_result)

In [ ]:
collection_id='Your_collection_id'
def generate_soil_data(num_sensors=3, num_readings=24):
    """Generate soil sensor data"""
    data = []
    base_time = datetime.now() - timedelta(hours=24)

    for sensor_num in range(1, num_sensors + 1):
        sensor_id = f"SOIL_{sensor_num:03d}"

        for hour in range(num_readings):
            timestamp = base_time + timedelta(hours=hour)

            reading = {
                "key": sensor_id,
                "timestamp": timestamp.strftime("%Y-%m-%dT%H:%M:%S"),
                "soil_moisture": round(random.uniform(30, 60), 1),
                "ph_level": round(random.uniform(6.0, 7.5), 1),
                "temperature": round(random.uniform(15, 25), 1),
                "nitrogen": round(random.uniform(30, 60), 1),
                "phosphorus": round(random.uniform(8, 20), 1),
                "potassium": round(random.uniform(120, 220), 1),
                "battery_level": round(random.uniform(70, 100), 1)
            }
            data.append(reading)

    return data

def send_data(project_id, collection_id, write_key, data):
    """Send data to collection"""
    url = f"{BASE_URL}/projects/{project_id}/collections/{collection_id}/send_data"
    headers = {"X-API-Key": write_key}
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        print(f"✅ Sent {len(data)} records")
        return True
    else:
        print(f"Failed to send data: {response.text}")
        return False

# Generate and send data
soil_data = generate_soil_data(num_sensors=3, num_readings=24)
send_data(PROJECT_ID, collection_id, WRITE_KEY, soil_data)

In [ ]:
def get_data(project_id, collection_id, read_key, filters=None):
    """Get data from collection with filters"""
    url = f"{BASE_URL}/projects/{project_id}/collections/{collection_id}/get_data"
    headers = {"X-API-Key": read_key}

    params = {}
    if filters:
        params["filters"] = json.dumps(filters)

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        print(f"✅ Retrieved {len(data)} records")
        return data
    else:
        print(f"Failed to get data: {response.text}")
        return []

# Example 1: Get all data
print("\n📥 Getting all data...")
all_data = get_data(PROJECT_ID, collection_id, READ_KEY)
print(all_data)

In [ ]:
# Example 2: Get data for specific sensor
print("\n🔍 Filter by sensor SOIL_001...")
sensor_filter = [
    {
        "property_name": "key",
        "operator": "eq",
        "property_value": "SOIL_001"
    }
]
sensor_data = get_data(PROJECT_ID, collection_id, READ_KEY, sensor_filter)

# Example 3: Get high moisture readings
print("\n💧 Filter for high moisture (>45%)...")
moisture_filter = [
    {
        "property_name": "soil_moisture",
        "operator": "gt",
        "property_value": 45
    }
]
high_moisture = get_data(PROJECT_ID, collection_id, READ_KEY, moisture_filter)
# Example 4: Complex filter - low moisture AND low battery
print("\n⚠️ Filter for low moisture AND low battery...")
complex_filter = [
    {
        "property_name": "soil_moisture",
        "operator": "lt",
        "property_value": 35
    },
    {
        "property_name": "battery_level",
        "operator": "lt",
        "property_value": 80
    }
]
alert_data = get_data(PROJECT_ID, collection_id, READ_KEY, complex_filter)
print(alert_data)

In [ ]:
## Step 4: Get Statistics

def get_statistics(project_id, collection_id, read_key, attribute, stat="avg", filters=None):
    """Get statistics for attribute"""
    url = f"{BASE_URL}/projects/{project_id}/collections/{collection_id}/statistics"
    headers = {"X-API-Key": read_key}

    params = {
        "attribute": attribute,
        "stat": stat,
        "interval": "every_24_hours"
    }

    if filters:
        params["filters"] = json.dumps(filters)

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        stats = response.json()
        print(f"✅ {stat} for {attribute}: {stats}")
        return stats
    else:
        print(f"Failed to get statistics: {response.text}")
        return {}

# Calculate statistics
print("\n📈 Getting statistics...")

# Average moisture
get_statistics(PROJECT_ID, collection_id, READ_KEY, "soil_moisture", "avg")

# Maximum temperature
get_statistics(PROJECT_ID, collection_id, READ_KEY, "temperature", "max")

# Minimum battery level
get_statistics(PROJECT_ID, collection_id, READ_KEY, "battery_level", "min")